In [1]:
#import libraries
from bs4 import BeautifulSoup as bs
import requests 
import time
import datetime
import pandas as pd
import pprint

import smtplib

In [2]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

#convert to beautiful soup object
soup = bs(page.content)

#print out the HTML
contents = soup.prettify()
print(contents)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-disabled" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Walt Disney Pictures films - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zeb

In [3]:
#find all elements in the page
td_elements = soup.find_all('td')
print(td_elements)

[<td><i><a href="/wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)" title="Snow White and the Seven Dwarfs (1937 film)">Snow White and the Seven Dwarfs</a></i>
</td>, <td><span data-sort-value="000000001937-12-21-0000" style="white-space:nowrap">December 21, 1937</span>
</td>, <td>first film to be distributed by <a href="/wiki/RKO_Pictures" title="RKO Pictures">RKO Radio Pictures</a>
</td>, <td><i><a href="/wiki/Pinocchio_(1940_film)" title="Pinocchio (1940 film)">Pinocchio</a></i>
</td>, <td><span data-sort-value="000000001940-02-07-0000" style="white-space:nowrap">February 7, 1940</span>
</td>, <td>
</td>, <td><i><a href="/wiki/Fantasia_(1940_film)" title="Fantasia (1940 film)">Fantasia</a></i>
</td>, <td><span data-sort-value="000000001940-11-13-0000" style="white-space:nowrap">November 13, 1940</span>
</td>, <td>anthology film
</td>, <td><i><a href="/wiki/The_Reluctant_Dragon_(1941_film)" title="The Reluctant Dragon (1941 film)">The Reluctant Dragon</a></i>
</td>, <td><span data-so

In [4]:
#Just checking
# iterate over each <td> element and extract the title
for td in td_elements:
    title_element = td.find('a')
    if title_element:
        title = title_element.get('title')
        print(title)

Snow White and the Seven Dwarfs (1937 film)
RKO Pictures
Pinocchio (1940 film)
Fantasia (1940 film)
The Reluctant Dragon (1941 film)
Dumbo
Bambi
Saludos Amigos
Victory Through Air Power (film)
United Artists
The Three Caballeros
Make Mine Music
Song of the South
Fun and Fancy Free
Melody Time
So Dear to My Heart
The Adventures of Ichabod and Mr. Toad
Cinderella (1950 film)
Treasure Island (1950 film)
Alice in Wonderland (1951 film)
The Story of Robin Hood (film)
Peter Pan (1953 film)
The Sword and the Rose
The Living Desert
Buena Vista Pictures Distribution
Rob Roy: The Highland Rogue
RKO Pictures
The Vanishing Prairie
20,000 Leagues Under the Sea (1954 film)
Davy Crockett: King of the Wild Frontier (film)
Disney anthology television series
Lady and the Tramp
The African Lion
The Littlest Outlaw
The Great Locomotive Chase
Davy Crockett and the River Pirates
Disney anthology television series
Secrets of Life
Westward Ho the Wagons!
Johnny Tremain (film)
Perri (film)
Old Yeller (film)
Th

In [5]:
# iterate over each <td> element
for td in td_elements:
    # find the first <a> element within the <td> element
    link_element = td.find('a')

    # check if <a> element exists
    if link_element:
        # get the title and href attributes
        title = link_element.get('title')
        href = link_element.get('href')

        # print the title and link
        print("Title:", title)
        print("Link:", href)

Title: Snow White and the Seven Dwarfs (1937 film)
Link: /wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)
Title: RKO Pictures
Link: /wiki/RKO_Pictures
Title: Pinocchio (1940 film)
Link: /wiki/Pinocchio_(1940_film)
Title: Fantasia (1940 film)
Link: /wiki/Fantasia_(1940_film)
Title: The Reluctant Dragon (1941 film)
Link: /wiki/The_Reluctant_Dragon_(1941_film)
Title: Dumbo
Link: /wiki/Dumbo
Title: Bambi
Link: /wiki/Bambi
Title: Saludos Amigos
Link: /wiki/Saludos_Amigos
Title: Victory Through Air Power (film)
Link: /wiki/Victory_Through_Air_Power_(film)
Title: United Artists
Link: /wiki/United_Artists
Title: The Three Caballeros
Link: /wiki/The_Three_Caballeros
Title: Make Mine Music
Link: /wiki/Make_Mine_Music
Title: Song of the South
Link: /wiki/Song_of_the_South
Title: Fun and Fancy Free
Link: /wiki/Fun_and_Fancy_Free
Title: Melody Time
Link: /wiki/Melody_Time
Title: So Dear to My Heart
Link: /wiki/So_Dear_to_My_Heart
Title: The Adventures of Ichabod and Mr. Toad
Link: /wiki/The_Advent

In [6]:
# Function to extract actual content value
def actual_content_value(row_data):
    # Remove all sup tags
    for sup in row_data.find_all("sup"):
        sup.decompose()
    # Check if row has a list <li> in the HTML
    if row_data.find("li"):
        return [li.get_text(strip=True).replace('\n', ' ') for li in row_data.find_all("li")]
    #check for <br> and split accordingly
    elif row_data.find("br"):
        return [text for text in row_data.stripped_strings]
    else:
        return row_data.find("td").get_text(strip=True).replace('\n', '').replace('\xa0', ' ')
def get_info_box(url):
    page = requests.get(url)

    # convert to beautiful soup object
    soup = bs(page.content)

    # extract all the rows
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all("tr")

    # Create an empty dictionary to store the data
    movie_info = {}

    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find("th").get_text().strip()
        elif index == 1:
            continue
        else:
            content_key = row.find("th").get_text(" ", strip=True).replace('\n', ' ')
            content_value = actual_content_value(row)
            movie_info[content_key] = content_value

            
                # Replace '\xa0' with a space in the release date value
        release_date_key = 'Release date' if 'Release date' in movie_info else 'Release dates'
        modified_release_date = movie_info.get(release_date_key, [''])[0].replace('\xa0', ' ')

        # Update the release date value in the dictionary
        movie_info[release_date_key] = [modified_release_date]

        # Check if the key is 'Production company' or 'Production companies'
        production_key = 'Production company' if 'Production company' in movie_info else 'Production companies'

        # Remove the space between the items in 'Production companies' list
        production_value = movie_info.get(production_key, [])
        if isinstance(production_value, list):
            production_companies = [company.replace(' ', '') for company in production_value]
            movie_info[production_key] = production_companies
    pprint.pprint(movie_info)
    return movie_info


In [7]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup = bs(page.content)
all_td_elements = soup.find_all('td')

base_path = "https://www.wikipedia.org/"

movie_info_list = []

for td in all_td_elements:
    link_element = td.find('a')

    if link_element:
        title = link_element.get('title')
        href = link_element.get('href')

        if title and title != 'None' and title != '20th Digital Studio' and not title.startswith('List') and not title.startswith('Walt'):
            print("Title:", title) 
            print("Link:", href)
            try:
                relative_path = href
                full_path = base_path + relative_path

                movie_info_list.append(get_info_box(full_path))

            except Exception as e:
                print(href)
                print(e)

Title: Snow White and the Seven Dwarfs (1937 film)
Link: /wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)
{'Based on': ['Based on', 'Snow White', 'by the', 'Brothers Grimm'],
 'Box office': '$418 million',
 'Budget': '$1.5 million',
 'Country': 'United States',
 'Directed by': ['David Hand',
                 'Perce Pearce',
                 'William Cottrell',
                 'Larry Morey',
                 'Wilfred Jackson',
                 'Ben Sharpsteen'],
 'Distributed by': 'RKO Radio Pictures',
 'Language': 'English',
 'Music by': ['Frank Churchill', 'Leigh Harline', 'Paul Smith'],
 'Produced by': 'Walt Disney',
 'Production companies': [],
 'Production company': ['Production', 'company', 'WaltDisneyProductions'],
 'Release dates': ['December 21, 1937(1937-12-21)(Carthay Circle Theatre)'],
 'Running time': '83 minutes',
 'Starring': ['Adriana Caselotti',
              'Roy Atwell',
              'Pinto Colvig',
              'Otis Harlan',
              'Scotty Mattraw',
     

KeyboardInterrupt: 

In [8]:
len(movie_info_list)

1

In [ ]:
save_path = r"G:\my_projects\disney\disney_data_cleaned.json"
load_path = r"G:\my_projects\disney\disney_data_cleaned.json"

In [ ]:
#save and reload data
import json

def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
# Saving data to the specified location
save_path = r"G:\my_projects\disney\disney_data_cleaned.json"
save_data(save_path, movie_info_list)
    

In [9]:
import json

def load_data(title):
    with open(title, 'r', encoding='utf-8') as f:
        return json.load(f)

# Loading data from the specified location
load_path = r"G:\my_projects\disney\disney_data_cleaned.json"
movie_info_list = load_data(load_path)


In [10]:
movie_info_list[-10]

{'title': 'Tinker Bell\nDisney Fairies film series',
 'Release dates': ['Release dates'],
 'Production companies': [],
 'Directed by': ['Bradley Raymond (1 & 3)',
  'Klay Hall (2)',
  'Peggy Holmes (4 & 5)',
  'Steve Loter (6)'],
 'Starring': ['Mae Whitman', 'Lucy Liu', 'Raven-Symoné', '(More)'],
 'Music by': 'Joel McNeely',
 'Production company': ['Production', 'company', 'DisneyToonStudios'],
 'Distributed by': ['Distributed by',
  'Walt Disney Studios',
  'Home Entertainment'],
 'Running time': ['Running time', '468 minutes', '(6 films)'],
 'Country': 'United States',
 'Language': 'English'}

In [ ]:
for movie_info in movie_info_list:
    if 'Running time' in movie_info:
        running_time_data = movie_info['Running time']
        running_time = None  # Initialize running_time as None
        
        # Check the type of 'Running time' value
        if isinstance(running_time_data, list):
            for item in running_time_data:
                if isinstance(item, int):
                    running_time = item
                    break  # Stop searching once an integer item is found
                elif isinstance(item, str) and (' minutes' in item or 'min' in item):
                    running_time_str = item
                    running_time = int(running_time_str.split(' ')[0])
                    break  # Stop searching once a valid string item is found
        elif isinstance(running_time_data, int):
            running_time = running_time_data

        # Update 'Running time' with the extracted value or None if not found
        movie_info['Running time'] = running_time

# Now, the 'Running time' values in movie_info_list have been updated to integers or None if not found
for movie_info in movie_info_list:
    print(movie_info.get('Running time'))

In [ ]:
movie_info_list[-9]

In [ ]:
for movie_info in movie_info_list:
    if 'Budget' in movie_info:
        budget_value = movie_info['Budget']
        print(budget_value)